## Reading generated images

In [ ]:
import cv2
import json
import time
import requests
import numpy as np
from matplotlib import pyplot as plt
BASE_URL = "https://dgmd-s17-assets.s3.amazonaws.com/train/generated-text-images/"

In [ ]:
with open('merged.json', 'r') as rf:
    data = json.load(rf)

prompts = data["prompts"]
sources = data["images"]

In [ ]:
for src in sources:
    image_url = BASE_URL + src["file_name"]
    resp = requests.get(image_url, stream=True).raw
    arr = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)[:,:,::-1]
    plt.imshow(img)
    plt.show()
    time.sleep(0.1)